In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
## check null values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df.shape

(20800, 5)

In [9]:
df=df.dropna()

In [10]:
df.shape

(18285, 5)

In [11]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [13]:
## label - is output feature where 1 - fake news ; 0 - not fake news so it is dependent feature, dropping label
X=df.drop('label', axis=1)

In [14]:
X

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [15]:
## Y being output feature
y= df['label']

In [16]:
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [18]:
## to check wheather dataset is balanced or not
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [19]:
import tensorflow as tf

In [20]:
tf.__version__

'2.15.0'

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [22]:
## vocab size in dictionary
voc_size=5000

In [23]:
## making a copy of dataset
messages = X.copy()

In [24]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [26]:
import nltk
import re
from nltk.corpus import stopwords

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [37]:
## since we removed nan values ; we will reset the data according to indexes
messages.reset_index(inplace=True)

In [38]:
## Dataset Preprocessing- using stemming as we are doing sentiment analysis ;
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus= []
for i in range(0,len(messages)):
  print(i)
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [39]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [41]:
## one hot representation- we will get indexes of all words in title
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2354, 3247, 3551, 471, 3688, 2037, 2573, 4899, 3845, 3870],
 [370, 4170, 4707, 4811, 1711, 593, 4826],
 [273, 2264, 80, 2874],
 [463, 606, 1256, 1770, 3255, 2378],
 [3795, 1711, 3297, 3647, 3650, 484, 1711, 3152, 4191, 3896],
 [3064,
  4413,
  1697,
  2307,
  1842,
  2542,
  4400,
  3878,
  8,
  3127,
  4927,
  2625,
  2821,
  882,
  4826],
 [4979, 1428, 4176, 1966, 3652, 139, 2021, 1054, 872, 1098, 3129],
 [1000, 2939, 1264, 1809, 3487, 2409, 2542, 1057, 872, 1098, 3129],
 [2570, 243, 1582, 4888, 140, 4748, 4852, 2933, 2542, 51],
 [1805, 2585, 4868, 3303, 2996, 2830, 3803, 4430],
 [2866, 950, 1100, 734, 4460, 2660, 1185, 1288, 246, 663, 1440],
 [1770, 4371, 3688, 4748, 2542, 3487],
 [1663, 3852, 1987, 4642, 4115, 1686, 3096, 3128, 1210],
 [1181, 2556, 2021, 2820, 2433, 2415, 3609, 872, 1098, 3129],
 [1311, 2042, 3138, 2375, 946, 872, 1098, 3129],
 [4849, 2542, 3281, 663, 3780, 10, 2765, 2792, 720, 1996],
 [24, 2116, 4170],
 [1461, 3515, 2027, 1837, 2542, 4593, 3537, 4826],
 [811, 57

In [ ]:
## passing these indexes to embedding layer and then later to bidirectional LSTM



Embedding represenation


In [42]:
sent_length=20
## before applying to embedding layer - we need to do the padding ; to make it to equal length

embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4899, 3845, 3870],
       [   0,    0,    0, ..., 1711,  593, 4826],
       [   0,    0,    0, ..., 2264,   80, 2874],
       ...,
       [   0,    0,    0, ...,  872, 1098, 3129],
       [   0,    0,    0, ...,  471,  786,  133],
       [   0,    0,    0, ...,  731, 1728, 1754]], dtype=int32)

In [44]:
## creating model- embedding_vector_features= means embedding layer thickness ; ( shud be more than sent_length)
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
##model.add(LSTM(100))
## instead of LSTM - we will add bidrictional
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [45]:
## for train and test- we will convert the embedded layer into an array ; along with independent feature

import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [46]:
## train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [49]:
## Model Training; batch size = depends on RAM
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 22s 46ms/step - loss: 0.2531 - accuracy: 0.8889 - val_loss: 0.2119 - val_accuracy: 0.9077
Epoch 2/10
383/383 [==============================] - 18s 46ms/step - loss: 0.1328 - accuracy: 0.9477 - val_loss: 0.2021 - val_accuracy: 0.9153
Epoch 3/10
383/383 [==============================] - 17s 44ms/step - loss: 0.0864 - accuracy: 0.9676 - val_loss: 0.2274 - val_accuracy: 0.9137
Epoch 4/10
383/383 [==============================] - 17s 44ms/step - loss: 0.0566 - accuracy: 0.9802 - val_loss: 0.2922 - val_accuracy: 0.9145
Epoch 5/10
383/383 [==============================] - 23s 60ms/step - loss: 0.0353 - accuracy: 0.9886 - val_loss: 0.3699 - val_accuracy: 0.9150
Epoch 6/10
383/383 [==============================] - 16s 42ms/step - loss: 0.0216 - accuracy: 0.9931 - val_loss: 0.4474 - val_accuracy: 0.9114
Epoch 7/10
383/383 [==============================] - 16s 43ms/step - loss: 0.0127 - accuracy: 0.9966 - val_loss: 0.4879 - val_accuracy:

In [ ]:
## accuracy getting increased with each epoch ; loss function getting minimized

In [50]:
## predicting data using confusion matrix
y_pred=model.predict(X_test)
y_pred

189/189 [==============================] - 4s 13ms/step


array([[9.9976754e-01],
       [1.4942393e-04],
       [5.9741564e-05],
       ...,
       [4.1206462e-05],
       [9.9993825e-01],
       [9.9998784e-01]], dtype=float32)

In [52]:
## using numpy to replace pred 0.5 with 1 otherwise all others 0
y_pred=np.where(y_pred>=0.5,1,0)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [55]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test,y_pred)

array([[3139,  280],
       [ 268, 2348]])

In [57]:
accuracy_score(y_test,y_pred)

0.9091963545981773